In [1]:
import pandas as pd
import torch
import os

from torch.utils.data import Dataset
from transformers import (BertTokenizer, BertForSequenceClassification, Trainer,
                          TrainingArguments, BertPreTrainedModel)
# from simpletransformers.language_modeling import LanguageModelingModel
from sklearn.metrics import accuracy_score, f1_score


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0))

True
NVIDIA GeForce RTX 3060
_CudaDeviceProperties(name='NVIDIA GeForce RTX 3060', major=8, minor=6, total_memory=12050MB, multi_processor_count=28)


In [3]:
MODEL_NAME = '/home/abdullah/Code/dl/lt_bert/best_models/best_tiny_bert'
TRAIN_FILE_LOC = '/home/abdullah/Code/dl/bnlp-resources/sentiment/multichannel_bsentiment/multichannel_bsentiment_train.tsv'
TEST_FILE_LOC = '/home/abdullah/Code/dl/bnlp-resources/sentiment/multichannel_bsentiment/multichannel_bsentiment_test.tsv'
EVAL_FILE_LOC = '/home/abdullah/Code/dl/bnlp-resources/sentiment/multichannel_bsentiment/multichannel_bsentiment_dev.tsv'


In [4]:
def tsv_to_text(tsv_file_loc):
    file_name = tsv_file_loc.split("/")[-1].split(".")[0]
    txt_name = tsv_file_loc.replace(".tsv", ".txt")
    txt_name = txt_name.replace("split_merged", "texts")

    if os.path.exists(txt_name):
        return [txt_name, file_name]

    df = pd.read_csv(tsv_file_loc, sep="\t")

    for txt in df["text"]:
        with open(txt_name, "a", encoding="utf8") as f:
            f.writelines(txt + "\n")
    return [txt_name, file_name]


tsv_to_text(TRAIN_FILE_LOC)


['/home/abdullah/Code/dl/bnlp-resources/sentiment/multichannel_bsentiment/multichannel_bsentiment_train.txt',
 'multichannel_bsentiment_train']

In [5]:
def tsv_to_df(csv_file_loc):
    df = pd.read_csv(csv_file_loc, sep='\t')
    # remove id column
    df = df.drop(columns=['id'])

    # replace neutral with 0, positive with 1, negative with 2 in class_label column
    df.loc[df['class_label'] == 'Neutral', 'class_label'] = 0
    df.loc[df['class_label'] == 'Positive', 'class_label'] = 1
    df.loc[df['class_label'] == 'Negative', 'class_label'] = 2

    return df


train_df = tsv_to_df(TEST_FILE_LOC)

texts = train_df['text'].tolist()
print(train_df)


                                                   text class_label
0     এমন কোনও রেস্টুরেন্ট হবে না যা আমি ফিরে আসব,যদ...           1
1     এমন কোনও রেস্টুরেন্ট হবে না যা আমি ফিরে আসব,যদ...           1
2     একটি সন্দেহ ছাড়া Orsay, NYC মধ্যে খাঁটি ফরাসি...           1
3                           কর্মচারী কোন অর্থহীনতা হয়।           1
4     চীনা ডেজার্টগুলি পরিচিত মিষ্টি ছাড়াও স্বাস্থ্...           1
...                                                 ...         ...
1527  সেখানে কোন পরিবেশ ছিল না,সেবা ছিল অনুপযুক্ত,এব...           2
1528  সেখানে কোন পরিবেশ ছিল না,সেবা ছিল অনুপযুক্ত,এব...           2
1529  সেখানে কোন পরিবেশ ছিল না,সেবা ছিল অনুপযুক্ত,এব...           2
1530             নির্দিষ্টভাবে এই এক আমার অন্তত প্রিয়।           2
1531  মাছ দুর্ভাগ্যবশত অতীতের মত অতীত হিসাবে প্রস্তু...           2

[1532 rows x 2 columns]


In [6]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }


In [7]:
def f1_calculator(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  f1 = f1_score(labels, preds, average='weighted')
  return {
      'f1': f1,
  }


In [8]:
train_list = tsv_to_text(TRAIN_FILE_LOC)
test_list = tsv_to_text(TEST_FILE_LOC)
eval_list = tsv_to_text(EVAL_FILE_LOC)


In [9]:
os.environ["WANDB_DISABLED"] = "true"

train_df = tsv_to_df(TRAIN_FILE_LOC)
test_df = tsv_to_df(TEST_FILE_LOC)
eval_df = tsv_to_df(EVAL_FILE_LOC)


class TINYDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(value[idx])
                for key, value in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

train_encodings = tokenizer(train_df['text'].tolist(
), padding=True, truncation=True, max_length=512)
test_encodings = tokenizer(test_df['text'].tolist(
), padding=True, truncation=True, max_length=512)
eval_encodings = tokenizer(eval_df['text'].tolist(
), padding=True, truncation=True, max_length=512)

train_dataset = TINYDataset(train_encodings, train_df['class_label'].tolist())
test_dataset = TINYDataset(test_encodings, test_df['class_label'].tolist())
eval_dataset = TINYDataset(eval_encodings, eval_df['class_label'].tolist())

model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3).to('cuda')
model.manual_seed = 14


Some weights of the model checkpoint at /home/abdullah/Code/dl/lt_bert/best_models/best_tiny_bert were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model c

## 1 Epoch Train

In [10]:
# training_args = TrainingArguments(
#     output_dir=f"temp",
#     num_train_epochs=1,
#     per_device_train_batch_size=48,
#     per_device_eval_batch_size=96,
#     warmup_steps=500,
#     learning_rate=5e-5,
#     weight_decay=0.01,
#     overwrite_output_dir=True,
#     logging_dir=f"temp/logs",
#     logging_steps=115,
#     save_steps=115,
#     load_best_model_at_end=True,
#     evaluation_strategy="steps",
#     seed=14,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset

# )

# trainer.train()


Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
***** Running training *****
  Num examples = 5510
  Num Epochs = 1
  Instantaneous batch size per device = 48
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 1
  Total optimization steps = 115
100%|██████████| 115/115 [00:05<00:00, 24.60it/s]***** Running Evaluation *****
  Num examples = 1378
  Batch size = 96


{'loss': 1.0801, 'learning_rate': 1.1500000000000002e-05, 'epoch': 1.0}


                                                 
100%|██████████| 115/115 [00:05<00:00, 24.60it/s]Saving model checkpoint to temp/checkpoint-115
Configuration saved in temp/checkpoint-115/config.json


{'eval_loss': 0.981864333152771, 'eval_runtime': 0.314, 'eval_samples_per_second': 4388.846, 'eval_steps_per_second': 47.774, 'epoch': 1.0}


Model weights saved in temp/checkpoint-115/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from temp/checkpoint-115 (score: 0.981864333152771).
100%|██████████| 115/115 [00:06<00:00, 17.01it/s]

{'train_runtime': 6.7684, 'train_samples_per_second': 814.08, 'train_steps_per_second': 16.991, 'train_loss': 1.08013159710428, 'epoch': 1.0}


TrainOutput(global_step=115, training_loss=1.08013159710428, metrics={'train_runtime': 6.7684, 'train_samples_per_second': 814.08, 'train_steps_per_second': 16.991, 'train_loss': 1.08013159710428, 'epoch': 1.0})

## 100 Epoch with 1 epoch model

In [12]:
MODEL_NAME = '/home/abdullah/Code/dl/lt_bert/best_models/1/1_beng_sentiment'
model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=3).to('cuda')
model.manual_seed = 14

for name, param in model.named_parameters():
    if 'bert.encoder.layer.0' in name:
        # print(name, param.requires_grad)
        param.requires_grad = False
# print("\n\n\n")
# for name, param in model.named_parameters():
#     print(name, param.requires_grad)

training_args = TrainingArguments(
    output_dir=f"temp",
    num_train_epochs=100,
    per_device_train_batch_size=192,
    per_device_eval_batch_size=384,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    overwrite_output_dir=True,
    logging_dir=f"temp/logs",
    logging_steps=400,
    save_steps=400,
    load_best_model_at_end=True,
    evaluation_strategy="steps",
    seed=14,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=f1_calculator,

)

trainer.train()

# print(cool.metrics)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,

)

trainer.train()


loading configuration file /home/abdullah/Code/dl/lt_bert/best_models/1/1_beng_sentiment/config.json
Model config BertConfig {
  "_name_or_path": "/home/abdullah/Code/dl/lt_bert/best_models/best_tiny_bert",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 

{'loss': 0.8272, 'learning_rate': 4e-05, 'epoch': 13.79}


                                                  
 14%|█▍        | 400/2900 [00:52<05:13,  7.96it/s]Saving model checkpoint to temp/checkpoint-400
Configuration saved in temp/checkpoint-400/config.json


{'eval_loss': 0.6185690760612488, 'eval_f1': 0.6946454687758926, 'eval_runtime': 0.3559, 'eval_samples_per_second': 3871.713, 'eval_steps_per_second': 11.239, 'epoch': 13.79}


Model weights saved in temp/checkpoint-400/pytorch_model.bin
 28%|██▊       | 800/2900 [01:47<04:33,  7.67it/s]***** Running Evaluation *****
  Num examples = 1378
  Batch size = 384


...


 97%|█████████▋| 2800/2900 [06:02<00:13,  7.57it/s]Saving model checkpoint to temp/checkpoint-2800
Configuration saved in temp/checkpoint-2800/config.json


{'eval_loss': 0.6031210422515869, 'eval_accuracy': 0.7402031930333817, 'eval_runtime': 0.3829, 'eval_samples_per_second': 3599.174, 'eval_steps_per_second': 10.448, 'epoch': 96.55}


Model weights saved in temp/checkpoint-2800/pytorch_model.bin
100%|█████████▉| 2899/2900 [06:17<00:00,  7.57it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from temp/checkpoint-800 (score: 0.5514978170394897).
100%|██████████| 2900/2900 [06:17<00:00,  7.69it/s]

{'train_runtime': 377.2071, 'train_samples_per_second': 1460.736, 'train_steps_per_second': 7.688, 'train_loss': 0.3702958889665275, 'epoch': 100.0}


TrainOutput(global_step=2900, training_loss=0.3702958889665275, metrics={'train_runtime': 377.2071, 'train_samples_per_second': 1460.736, 'train_steps_per_second': 7.688, 'train_loss': 0.3702958889665275, 'epoch': 100.0})